## Solving the Poisson Problem on a 4 dimensional semistructured P2 Fes

This Notebook is very similar to the P1 notebook. In general a more interessting application of the P2 element is in the [Navier-Stokes](/demos/linearised_Navier_Stokes/taylor-Hood.ipynb) Taylor-Hood discretisation

In [7]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import math
import numpy as np
from numpy import exp
from methodsnm.vectorspace import *
from methodsnm.fes import *
from netgen.csg import unit_cube
from ngsolve import Mesh,VOL,specialcf

Constructing a mesh

In [8]:
m = 2
T = 2
epsi = 0.1
ngmesh = Mesh(unit_cube.GenerateMesh(maxh=0.5))
mesh = UnstructuredHypertriangleMesh(T,ngmesh)
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

In [9]:
from methodsnm.forms import *
from methodsnm.formint import *
V = P2_Hypertriangle_Space(mesh)
V1 = P1_Hypertriangle_Space(mesh)
print("Number of dofs for P1Fes :",V1.ndof )
print("Number of dofs for P2Fes :",V.ndof , "As expected it is much larger than for the P1 element")
print(len(V.boundary_dofs()), "also their are more boundary dofs", len(mesh.bndry_vertices))

Number of dofs for P1Fes : 78
Number of dofs for P2Fes : 652 As expected it is much larger than for the P1 element
513 also their are more boundary dofs 72


Assembling the Biliniearform and the Linearform for P2 elements, and treating the boundary conditions acordingly

In [10]:
from numpy import cos, sin,pi
BF = BilinearForm(V)                      
BF += LaplaceIntegral()
BF.assemble()

u_ex = lambda x: cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])*cos(pi*x[3])
f    = lambda x: 4*pi**2*cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])*cos(pi*x[3])
f = GlobalFunction(f, mesh=mesh)

LFs = LinearForm(V)
LFs += SourceIntegral(f)
LFs.assemble()

all_dofs = list(range(V.ndof))
boundary_dofs = V.boundary_dofs()
freedofs = list_diff(all_dofs, boundary_dofs)

u_h = FEFunction(V)
u_h._set_P2(u_ex,boundary=True)

res =  LFs.vector - BF.matrix @ u_h.vector 
from methodsnm.solver import solve_on_freedofs
u_h.vector += solve_on_freedofs(BF.matrix, res, freedofs)

uex = FEFunction(V)
uex._set_P2(u_ex)
res = BF.matrix @ uex.vector - LFs.vector


In [11]:
from methodsnm.forms import compute_difference_L2
l2diff = compute_difference_L2(u_h, GlobalFunction(u_ex,mesh=mesh), mesh, intorder = 3)

print("l2diff =", l2diff)

l2diff = 0.033252096329250455


## Final remarks
So if you compare it to the P1 element you get better results, if you take the same mesh construction and in addition will also see a better convergence rate.